In [1]:
import pandas as pd
from matplotlib import pyplot as  plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
jug=pd.read_csv('players.csv', sep=';')
dfc=pd.read_csv('MaestroTirosKingsCup-2023.csv',sep=';')
defreg = pd.read_csv('defensa_regate_defic.csv',sep=';')
pas=pd.read_csv('MaestroPasesKingsCup-2023.csv', sep=';')

In [4]:
dfc['goal_y']=dfc['goal_y'].str.replace(',','.').astype('float')
dfc['goal_x']=dfc['goal_x'].str.replace(',','.').astype('float')
dfc['shot_x']=dfc['shot_x'].str.replace(',','.').astype('float')
dfc['shot_y']=dfc['shot_y'].str.replace(',','.').astype('float')
dfc['xG']=dfc['xG'].str.replace(',','.').astype('float')
pas['pass_x']=pas['pass_x'].str.replace(',','.').astype('float')
pas['pass_y']=pas['pass_y'].str.replace(',','.').astype('float')
pas['pass_end_x']=pas['pass_end_x'].str.replace(',','.').astype('float')

In [5]:
ind=jug.groupby('NOMBRES DEFINITIVOS').size().to_frame()
ind['Recuperaciones']=defreg[(defreg['Event']=='NRB')].groupby('Player').size()
ind['Tiros Bloqueados']=dfc.groupby('block_player_name').size()
ind['Intercepciones']=defreg[(defreg['Event']=='NBI')].groupby('Player').size()
ind['Faltas Realizadas']=defreg[(defreg['Event']=='FRA')].groupby('Player').size()
ind['Faltas Recibidas']=defreg[(defreg['Event']=='FR')].groupby('Player').size()
ind['Despejes']=defreg[(defreg['Event']=='ND')].groupby('Player').size()
ind['Entradas Éxito']=defreg[(defreg['Event']=='NJTS')].groupby('Player').size()
ind['Entradas Fallo']=defreg[(defreg['Event']=='NJTF')].groupby('Player').size()
ind['Regates Defendidos']=defreg[(defreg['Event']=='REE')].groupby('Player').size()
ind['Veces Regateado']=defreg[(defreg['Event']=='REF')].groupby('Player').size()
ind['Duelos Aereos Ganados']=defreg[(defreg['Event']=='DAG')].groupby('Player').size()
ind['Duelos Aereos Perdidos']=defreg[(defreg['Event']=='DAF')].groupby('Player').size()
ind['Errores_que_provocan_el_tiro_del_rival']=defreg[(defreg['Event']=='EPT')].groupby('Player').size()
ind['Pases Clave']=dfc.groupby('pass_player_name').size()
ind['Goles Marcados No Penalti']=dfc[(dfc['shot_outcome']=='Goal')&(dfc['situation']!='Penalty')&(dfc['situation']!='ShootOut')].groupby(['player_name']).size()
ind['Goles Marcados Penalti']=dfc[(dfc['shot_outcome']=='Goal')&((dfc['situation']=='Penalty')|(dfc['situation']=='ShootOut'))].groupby(['player_name']).size()
ind['Tiros a Puerta']=dfc[(dfc['goal_x']>0)&(dfc['goal_x']<6)&(dfc['goal_y']<2)].groupby('player_name').size()
ind['Goles/Tiros']=dfc[dfc['shot_outcome']=='Goal'].groupby('player_name').size()/dfc.groupby('player_name').size()
ind['Regates Completados']=defreg[(defreg['Event']=='RC')].groupby('Player').size()
ind['Regates Fallados']=defreg[(defreg['Event']=='RF')].groupby('Player').size()
ind['xG']=dfc.groupby('player_name')['xG'].sum()
ind=ind.fillna(0)
ind['Goles - NpxG']=ind['Goles Marcados No Penalti']-dfc[(dfc['situation']!='Penalty')&(dfc['situation']!='ShootOut')].groupby('player_name')['xG'].sum()
ind['Asistencias']=dfc[(dfc['shot_outcome']=='Goal')&(dfc['is_assist']=='VERDADERO')].groupby(['pass_player_name']).size()
ind['xG/Tiro']=ind['xG']/dfc.groupby('player_name').size()
ind['Recuperaciones Ofensivas']=dfc[dfc['previous_action']=='Recovery'].groupby('player_name').size()
ind['Carry Ofensiva']=dfc[dfc['previous_action']=='Carry'].groupby('player_name').size()
ind['Regate Ofensiva']=dfc[dfc['previous_action']=='Dribbling'].groupby('player_name').size()
ind['Pases Exito']=pas[pas['is_successful']=='VERDADERO'].groupby('player_name').size()
ind['Pases Fallo']=pas[pas['is_successful']=='FALSO'].groupby('player_name').size()
ind['Pases Acierto Presion']=pas[(pas['is_successful']=='VERDADERO')&(pas['is_under_pressure']=='VERDADERO')].groupby('player_name').size()
ind['Pases Acierto Ultimo Tercio']=pas[(pas['is_successful']=='VERDADERO')&(pas['pass_x']>=36)].groupby('player_name').size()
ind['Pases Hacia Delante']=pas[(pas['pass_end_x']>pas['pass_x']+4)].groupby('player_name').size()
ind=ind.fillna(0)

ind

,0,Recuperaciones,Tiros Bloqueados,Intercepciones,Faltas Realizadas,Faltas Recibidas,Despejes,Entradas Éxito,Entradas Fallo,Regates Defendidos,...,Asistencias,xG/Tiro,Recuperaciones Ofensivas,Carry Ofensiva,Regate Ofensiva,Pases Exito,Pases Fallo,Pases Acierto Presion,Pases Acierto Ultimo Tercio,Pases Hacia Delante
NOMBRES DEFINITIVOS,,,,,,,,,,,,,,,,,,,,,
A. Barragán,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. Biboum,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. De Castro,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. Fernández,1,25.0,5.0,20.0,5.0,3.0,7.0,2.0,0.0,2.0,...,0.0,0.076662,1.0,6.0,1.0,213.0,30.0,43.0,28.0,132.0
A. Garcia,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.053300,0.0,0.0,0.0,37.0,1.0,0.0,0.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xavi Boniquet,1,7.0,4.0,12.0,0.0,4.0,2.0,3.0,2.0,0.0,...,2.0,0.165625,1.0,2.0,2.0,90.0,8.0,8.0,25.0,41.0
Xavi Liebanas,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.076100,0.0,1.0,0.0,6.0,0.0,0.0,4.0,1.0
Xavi Molas,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
ind['Indice_defensivo']= (0.2* ind['Recuperaciones'] + 0.13* ind['Tiros Bloqueados'] + 0.17 * ind['Intercepciones']-
0.05 * ind['Faltas Realizadas']+
0.05 * ind['Despejes'] +
0.1 * ind['Regates Defendidos'] -
0.05 * ind['Veces Regateado']
#0.03 * ind['Errores_que_provocan_el_tiro_del_rival']+
+0.1 * ind['Entradas Éxito'] -
0.05 * ind['Entradas Fallo'] +
0.05 * ind['Duelos Aereos Ganados'] -
0.03 * ind['Duelos Aereos Perdidos'])
#0.03 * ind['Goles Marcados No Penalti']+
#0.07 * ind['Pases Clave']                        


min_value = 0
max_value = 10
ind=ind[ind['Indice_defensivo']!=0]
scaler = MinMaxScaler(feature_range=(min_value, max_value))

# Ajusta y transforma la columna 'Indice_defensivo'
ind['Indice_defensivo'] = scaler.fit_transform(ind[['Indice_defensivo']])

print(ind[['Indice_defensivo']].sort_values('Indice_defensivo',ascending=False)[:10])


                     Indice_defensivo
NOMBRES DEFINITIVOS                  
A. Fernández                10.000000
N. Pareja                    9.365079
C. Planas                    9.227513
N. Louah                     8.761905
I. González                  8.719577
M. Reyes                     8.645503
Joel Bañuls                  8.444444
A. Vives                     8.338624
M. Mantovani                 8.317460
V. Mongil                    7.587302


C:\Users\pablo\AppData\Local\Temp\ipykernel_11296\2274966651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind['Indice_defensivo']= (0.2* ind['Recuperaciones'] + 0.13* ind['Tiros Bloqueados'] + 0.17 * ind['Intercepciones']-


In [11]:
ind['Indice_ofensivo']= (0.15 * ind['Goles Marcados No Penalti'] + 0.05* ind['Goles Marcados Penalti'] + 0.12* ind['Regates Completados']- 
0.05 * ind['Regates Fallados']+
0.09 * ind['Asistencias'] +
0.08 * ind['Goles/Tiros'] +
0.12 * ind['Goles - NpxG']+
0.10 * ind['Tiros a Puerta']+
0.05 * ind['Faltas Recibidas'] +
0.03 * ind['Recuperaciones Ofensivas']+
#0.03 * ind['Regate Clave']+
0.03* ind['Carry Ofensiva']+                         
0.06* ind['xG/Tiro']+                     
0.07 * ind['Pases Clave'] )


min_value = 1
max_value = 10
ind=ind[ind['Indice_ofensivo']!=0]
ind['Indice_ofensivo'] = (
   min_value + (max_value - min_value) * 
    (ind['Indice_ofensivo'] - ind['Indice_ofensivo'].min()) /
 (ind['Indice_ofensivo'].max() - ind['Indice_ofensivo'].min())
)

print(ind[['Indice_ofensivo']].sort_values('Indice_ofensivo',ascending=False)[:10])


                     Indice_ofensivo
NOMBRES DEFINITIVOS                 
J. Verdú                   10.000000
Javi Espinosa               9.042971
Bueno                       8.663421
R. Carbó                    8.049172
Pablo Hernandez             7.579352
C. Ubón                     7.291812
C. Corvo                    7.170092
O. Boada                    6.790480
Fran Hernández              6.544906
Alejandro Ortega            6.496632


C:\Users\pablo\AppData\Local\Temp\ipykernel_11296\3227155975.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind['Indice_ofensivo'] = (


In [9]:
ind['Indice_creacion']= (0.3 * ind['Pases Exito'] - 0.1* ind['Pases Fallo'] + 0.15* ind['Pases Acierto Presion']- 
0.2 * ind['Pases Acierto Ultimo Tercio']+
0.1 * ind['Asistencias'] +
#0.1 * ind['Pases Hacia Delante'] +        
0.15 * ind['Pases Clave'] )


min_value = 0
max_value = 9.75

ind['Indice_creacion'] = (
   min_value + (max_value - min_value) * 
    (ind['Indice_creacion'] - ind['Indice_creacion'].min()) /
 (ind['Indice_creacion'].max() - ind['Indice_creacion'].min())
)

print(ind[['Indice_creacion']].sort_values('Indice_creacion',ascending=False)[:10])


                     Indice_creacion
NOMBRES DEFINITIVOS                 
Marc Valiente               9.750000
J. Verdú                    9.511667
V. Mongil                   8.662333
N. Pareja                   7.921333
Hugo Fraile                 7.514000
M. Reyes                    7.440333
Sergio Juste                7.037333
M. Mantovani                6.985333
G. Cichero                  6.924667
Javi Espinosa               6.903000
